In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import os 
os.chdir("/content/drive/My Drive/Sentiment Analysis Code")

In [0]:
import pandas as pd

In [0]:
data_set = pd.read_csv("labeledTrainData.tsv",header=0, delimiter="\t", quoting=3)

In [7]:
data_set.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [28]:
new_data_set = data_set
new_data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [0]:
import string
from nltk.corpus import stopwords

In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
def text_process(review_msg):
    nopunc_review_msg = [char for char in review_msg if char not in string.punctuation]
    nopunc_review_msg = ''.join(nopunc_review_msg)
    
    return [word for word in nopunc_review_msg.split() if word.lower not in stopwords.words('english')]

In [32]:
new_data_set['review'].head(5).apply(text_process)

0    [With, all, this, stuff, going, down, at, the,...
1    [The, Classic, War, of, the, Worlds, by, Timot...
2    [The, film, starts, with, a, manager, Nicholas...
3    [It, must, be, assumed, that, those, who, prai...
4    [Superbly, trashy, and, wondrously, unpretenti...
Name: review, dtype: object

In [33]:
len(new_data_set['review'])

25000

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(new_data_set['review'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

142668


In [0]:
review_bow = bow_transformer.transform(new_data_set['review'])

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(review_bow)

In [0]:
review_tfidf = tfidf_transformer.transform(review_bow)

In [39]:
print(review_tfidf.shape)

(25000, 142668)


In [0]:
from sklearn.naive_bayes import MultinomialNB
review_detect_model = MultinomialNB().fit(review_tfidf, new_data_set['sentiment'])

In [0]:
all_predictions = review_detect_model.predict(review_tfidf)

In [42]:
from sklearn.metrics import classification_report
print (classification_report(new_data_set['sentiment'], all_predictions))

              precision    recall  f1-score   support

           0       0.91      0.94      0.92     12500
           1       0.94      0.90      0.92     12500

   micro avg       0.92      0.92      0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000



In [0]:
from sklearn.metrics import confusion_matrix

In [44]:
print (confusion_matrix(new_data_set['sentiment'], all_predictions))

[[11782   718]
 [ 1205 11295]]
